In [7]:
import os
import subprocess
import numpy as np
import xarray as xr
import pandas as pd
from myfunc import timer
from myfunc import DirMan
import config
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"  # 显示所有输出



# configuration
# resolution = "0p1"
resolution = "500"
# name = "exp1"
# name = "exp2"
region = [-180,180,-60,90]
data_path = f'/tera11/zhwei/students/Xionghui/data/run/{resolution}/'
post_data_path = '/tera11/zhwei/students/Xionghui/data/'
shp_path = '/tera11/zhwei/students/Xionghui/data/Shp/'
fig_path = f'/home/xuxh22/stu01/Bedrock/fig/{resolution}/'
path = '/home/xuxh22/stu01/Bedrock/'

if resolution == "0p1":
    size = 0.1
elif resolution == "500":
    size = 0.0005



dir_man = DirMan(data_path)
dir_man.enter()

Entered /tera11/zhwei/students/Xionghui/data/run/500/


In [8]:
ds = xr.open_dataset(f'{data_path}diff.nc4')
current_diff = ds['et']
ds2 = xr.open_dataset(f'{data_path}SnowCover.nc4')
snowf = ds2['snowf']
ds3 = xr.open_dataset(f'{data_path}../0p1/Ssoil.nc4')
ssoil = ds3['Band1']
print(data_path)

/tera11/zhwei/students/Xionghui/data/run/500/


In [9]:
years = pd.date_range(start='2003-01-01', end='2020-01-01', freq='YS')

time_array = xr.DataArray(
    years,
    dims=['time'],
    coords={'time': years},
    name='time',
    attrs={
        'standard_name': 'time',
        'axis': 'T'
    }
)

print(time_array)
print(f"Length: {len(time_array)}")

<xarray.DataArray 'time' (time: 18)>
array(['2003-01-01T00:00:00.000000000', '2004-01-01T00:00:00.000000000',
       '2005-01-01T00:00:00.000000000', '2006-01-01T00:00:00.000000000',
       '2007-01-01T00:00:00.000000000', '2008-01-01T00:00:00.000000000',
       '2009-01-01T00:00:00.000000000', '2010-01-01T00:00:00.000000000',
       '2011-01-01T00:00:00.000000000', '2012-01-01T00:00:00.000000000',
       '2013-01-01T00:00:00.000000000', '2014-01-01T00:00:00.000000000',
       '2015-01-01T00:00:00.000000000', '2016-01-01T00:00:00.000000000',
       '2017-01-01T00:00:00.000000000', '2018-01-01T00:00:00.000000000',
       '2019-01-01T00:00:00.000000000', '2020-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2003-01-01 2004-01-01 ... 2020-01-01
Attributes:
    standard_name:  time
    axis:           T
Length: 18


In [10]:
# Obtain shape and duration
shape = current_diff.isel(time=0).shape
time_len = len(ds.time)

print(time_len)

# Initialize variables
current_cwd = np.zeros(shape) 
sr = np.zeros(shape)
# sbedrock = np.zeros(shape)

use_sbedrock_frequency = np.zeros((18, *shape))
use_sbedrock_cross_year = np.zeros((18, *shape))
use_sbedrock_time_duration = np.zeros(shape)
use_sbedrock_sum_time = np.zeros(shape)
use_sbedrock_time_mean_duration = np.zeros(shape)
use_sbedrock_time_max_duration = np.zeros(shape)

use_sbedrock_sum_duration = np.zeros(shape)
use_sbedrock_year_sum_duration = np.zeros((18,*shape))
use_sbedrock_first_day = np.zeros((18,*shape))
use_sbedrock_period = np.zeros((time_len, *shape))

print(ds['time'])

828
<xarray.DataArray 'time' (time: 828)>
array(['2003-01-01T00:00:00.000000000', '2003-01-09T00:00:00.000000000',
       '2003-01-17T00:00:00.000000000', ..., '2020-12-10T00:00:00.000000000',
       '2020-12-18T00:00:00.000000000', '2020-12-26T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2003-01-01 2003-01-09 ... 2020-12-26
Attributes:
    standard_name:  time
    axis:           T


In [11]:
for i in range(time_len):
    print(f"Processing time index: {i}")
    day_stt = 8*i+1-3*(i//46)+((i//46)+2)//4
    day_end = 8*(i+1)+1-3*((i+1)//46)+(((i+1)//46)+2)//4-1
    day_duration = day_end-day_stt+1
    j = i//46
    year = j+2003
    print(year)
    print(f"the period {i+1:3} day from {day_stt:4} to {day_end:4}")
    print(f"the period {i+1:3} day is {day_duration:1}")

    # Calculate current delta_tn, cwd and sr
    current_delta_tn = current_diff.isel(time=i).values * snowf.isel(time=i).values
    last_cwd = current_cwd
    # last_cwd_reset = last_cwd
    # if i%46==0:
    #     last_cwd_reset = 0
    current_cwd = np.where(current_delta_tn >= 0, current_cwd + current_delta_tn, 0)
    sr = np.maximum(sr, current_cwd)
   
    # Calculate the first day, duration and all time periods of using bedrock water  
    mask1 = current_cwd > ssoil
    mask2 = last_cwd > ssoil
    mask3 = last_cwd <= ssoil

    if i%46==0:
        use_sbedrock_cross_year[j,:,:] = np.where(mask1 & mask2, 1, 0)

    use_sbedrock_frequency[j,:,:] = np.where(mask1 & mask3, use_sbedrock_frequency[j,:,:]+1, use_sbedrock_frequency[j,:,:])
    
    use_sbedrock_sum_time = np.where(mask1 & mask3, use_sbedrock_sum_time+1, use_sbedrock_sum_time)
    use_sbedrock_sum_duration = np.where(mask1, use_sbedrock_sum_duration + day_duration, use_sbedrock_sum_duration)

    use_sbedrock_time_duration = np.where(mask1, use_sbedrock_time_duration + day_duration, 0)
    use_sbedrock_time_max_duration = np.where(use_sbedrock_time_duration>use_sbedrock_time_max_duration, use_sbedrock_time_duration, use_sbedrock_time_max_duration)
    
    use_sbedrock_year_sum_duration [j,:,:] = np.where(mask1, use_sbedrock_year_sum_duration[j,:,:]+day_duration, use_sbedrock_year_sum_duration[j,:,:])
    use_sbedrock_first_day[j,:,:] = np.where(mask1 & (use_sbedrock_first_day[j,:,:] == 0), day_stt, use_sbedrock_first_day[j,:,:])
    use_sbedrock_period[i, :, :] = np.where(mask1, 1, 0)

use_sbedrock_time_mean_duration = np.where(use_sbedrock_sum_time > 0,use_sbedrock_sum_duration/use_sbedrock_sum_time, np.nan)

Processing time index: 0
2003
the period   1 day from    1 to    8
the period   1 day is 8
Processing time index: 1
2003
the period   2 day from    9 to   16
the period   2 day is 8
Processing time index: 2
2003
the period   3 day from   17 to   24
the period   3 day is 8
Processing time index: 3
2003
the period   4 day from   25 to   32
the period   4 day is 8
Processing time index: 4
2003
the period   5 day from   33 to   40
the period   5 day is 8
Processing time index: 5
2003
the period   6 day from   41 to   48
the period   6 day is 8
Processing time index: 6
2003
the period   7 day from   49 to   56
the period   7 day is 8
Processing time index: 7
2003
the period   8 day from   57 to   64
the period   8 day is 8
Processing time index: 8
2003
the period   9 day from   65 to   72
the period   9 day is 8
Processing time index: 9
2003
the period  10 day from   73 to   80
the period  10 day is 8
Processing time index: 10
2003
the period  11 day from   81 to   88
the period  11 day is 

/tmp/ipykernel_1838338/690095211.py:41: RuntimeWarning: invalid value encountered in divide
  use_sbedrock_time_mean_duration = np.where(use_sbedrock_sum_time > 0,use_sbedrock_sum_duration/use_sbedrock_sum_time, np.nan)


In [12]:
output_ds = xr.Dataset({'Sr': (('lat', 'lon'), sr)},
                    coords={'lat': ds['lat'], 'lon': ds['lon']})
output_ds.to_netcdf(f'{data_path}Sr_tmp1.nc4')

output_ds1 = xr.Dataset({'Sbedrock': (('lat', 'lon'), (sr-ssoil).data)},
                    coords={'lat': ds['lat'], 'lon': ds['lon']})
output_ds1.to_netcdf(f'{data_path}Sbedrock_tmp1.nc4')


output_ds1 = xr.Dataset({'Duration': (('lat', 'lon'), use_sbedrock_time_max_duration)},
                    coords={'lat': ds['lat'], 'lon': ds['lon']})
output_ds1.to_netcdf(f'{data_path}S_time_max_duration_tmp1.nc4')

output_ds1 = xr.Dataset({'Duration': (('lat', 'lon'), use_sbedrock_time_mean_duration)},
                    coords={'lat': ds['lat'], 'lon': ds['lon']})
output_ds1.to_netcdf(f'{data_path}S_time_mean_duration_tmp1.nc4')

output_ds1 = xr.Dataset({'Frequency': (('lat', 'lon'), use_sbedrock_sum_time)},
                    coords={'lat': ds['lat'], 'lon': ds['lon']})
output_ds1.to_netcdf(f'{data_path}S_sum_Frequency_tmp1.nc4')

output_ds1 = xr.Dataset({'Duration': (('lat', 'lon'), use_sbedrock_sum_duration)},
                    coords={'lat': ds['lat'], 'lon': ds['lon']})
output_ds1.to_netcdf(f'{data_path}S_sum_duration_tmp1.nc4')



output_ds2 = xr.Dataset({'Frequency': (('time','lat', 'lon'), use_sbedrock_frequency)},
                    coords={'time': time_array,'lat': ds['lat'], 'lon': ds['lon']})
output_ds2.to_netcdf(f'{data_path}S_Frequency_tmp1.nc4')

output_ds3 = xr.Dataset({'Cross_Year': (('time','lat', 'lon'), use_sbedrock_cross_year)},
                    coords={'time': time_array,'lat': ds['lat'], 'lon': ds['lon']})
output_ds3.to_netcdf(f'{data_path}S_CY_tmp1.nc4')

output_ds4 = xr.Dataset({'Duration': (('time','lat', 'lon'), use_sbedrock_year_sum_duration)},
                    coords={'time': time_array,'lat': ds['lat'], 'lon': ds['lon']})
output_ds4.to_netcdf(f'{data_path}S_Duration_tmp1.nc4')

output_ds5 = xr.Dataset({'First_Day': (('time','lat', 'lon'), use_sbedrock_first_day)},
                    coords={'time': time_array,'lat': ds['lat'], 'lon': ds['lon']})
output_ds5.to_netcdf(f'{data_path}S_FD_tmp1.nc4')


output_ds6 = xr.Dataset({'Period': (('time', 'lat', 'lon'), use_sbedrock_period)},
                    coords={'time': ds['time'], 'lat': ds['lat'], 'lon': ds['lon']})
output_ds6.to_netcdf(f'{data_path}S_Period_tmp1.nc4')